In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute,QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram, array_to_latex
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction

from qiskit.circuit.library import QFT
from numpy import pi
from scipy.signal import find_peaks
from time import time

#from qiskit.providers.aer.library import save_statevector
from qiskit.providers.aer.library import save_density_matrix

import warnings
warnings.filterwarnings('ignore')


class myqc(QuantumCircuit):
    def set_ct(self,cq,tq):
        self.cq=cq
        self.tq=tq

    # define Pauli gates    
    def paulis(self,qi,x):  
        if x==[0,0]:
            self.id(qi)
                     
        elif x==[0,1]:
            self.z(qi)
                     
        elif x==[1,0]:
            self.x(qi)
        elif x==[1,1]:
            self.y(qi)
    
    # define square root iswap gates
    def sqriswap(self,q0,q1):
        self.u3(np.pi/2,np.pi/2,-np.pi/2,q0)
        self.u3(2.6974496917414865,-np.pi/2,-np.pi/2,q1)
        self.cx(q0,q1)
        self.u3(3/4*np.pi,-np.pi,-np.pi/2,q0)
        self.u3(1.456908933230463,0.8728233028872143,-1.8815892705543913,q1)
        self.cx(q0,q1)
        self.u3(np.pi/2,np.pi/2,-np.pi,q0)
        self.u3(2.014939288643203,np.pi/2,np.pi/2,q1)


# the circuit of the unitary of interest
nq=6
U = myqc(nq)
for i in range(0,nq,2):
    #U.u3(np.pi/2,np.pi/2,-np.pi/2,i)
    U.u2(np.pi/2,-np.pi/2,i)
for i in range(1,nq,2):   
    U.u3(2.6974496917414865,-np.pi/2,-np.pi/2,i)
#U.barrier()
for i in range(0,nq,2):
    U.cx(i,i+1)
U.barrier()

for i in range(0,nq,2):
    U.u3(3/4*np.pi,-np.pi,-np.pi/2,i)
for i in range(1,nq,2):   
    U.u3(1.456908933230463,0.8728233028872143,-1.8815892705543913,i)
#U.barrier()
for i in range(0,nq,2):
    U.cx(i,i+1)
U.barrier()


for i in range(0,nq,2):   
    U.u3(2.014939288643203,np.pi/2,0,i)
for i in range(1,nq,2):
    U.u3(2.6974496917414865,-np.pi/2,-np.pi/2,i)
for i in range(1,nq,2):
    U.cx(i,(i+1)%nq)
U.barrier()

for i in range(0,nq,2):   
    U.u3(1.456908933230463,0.8728233028872143,-1.8815892705543913,i)
for i in range(1,nq,2):
    U.u3(3/4*np.pi,-np.pi,-np.pi/2,i)
for i in range(1,nq,2):
    U.cx(i,(i+1)%nq)
U.barrier()

for i in range(0,nq,2):   
    U.u3(2.014939288643203,np.pi/2,np.pi/2,i)
for i in range(1,nq,2):
    U.u3(np.pi/2,np.pi/2,-np.pi,i)
U.barrier()

# the true phases in this problem
num_p= int(nq/2)+1
true_p= np.sort([np.arccos(np.sin(4*pi/nq*k)**2) for k in range(int(num_p/2))]+ [-1*np.arccos(np.sin(4*pi/nq*k)**2) for k in range(int(num_p/2))])
#true_p= np.sort([true_p_tem[0]]+ list(true_p_tem[1:-1])*2+ [true_p_tem[-1]])
print(true_p)


# the function to generate the power of a unitary
def circ_rep(k,circ):
    num_qubits= circ.num_qubits
    U_rep=QuantumCircuit(num_qubits)
    for i in range(k):
        U_rep=U_rep.compose(U)
    
    return U_rep


# generate the full circuit for simulation by combining the initial state preparation and final measurement
def full_circ(floq_circ,mode='x',nq=6):
    qc= QuantumCircuit(nq)
    qc.h(0)
    qc.barrier()
    
    qc=qc.compose(floq_circ)
    # if mode=='y':
    #     qc.s(0)
    
    # qc.h(0)
    # qc.barrier()
    # qc.measure(0,0)
    qc.save_density_matrix()
    return qc


def sim_pre(nq=6):
    qc= QuantumCircuit(nq)
    qc.h(0)
    qc.save_density_matrix()
    return qc

def sim_circ(ini_state,nq=6):
    qc= QuantumCircuit(nq)
    qc.initialize(ini_state)
    #qc.h(0)
    qc.barrier()
    
    qc=qc.compose(U)
    # if mode=='y':
    #     qc.s(0)
    
    # qc.h(0)
    # qc.barrier()
    # qc.measure(0,0)
    qc.save_density_matrix()
    return qc


# # generate bare circuits with Pauli-X and Pauli-Y measurements for phase estimation 
# circ_rep_list=[circ_rep(k,U) for k in range(1,51)]
# full_circ_list=list(map(full_circ,circ_rep_list))
# # full_circ_my_list=list(map(lambda x:full_circ(x,mode='y'),circ_rep_list))


# define noise model for simulation
from qiskit_aer.noise  import NoiseModel, amplitude_damping_error, depolarizing_error, coherent_unitary_error,mixed_unitary_error, pauli_error
from scipy.optimize import curve_fit, minimize

def unitary_error_rx(theta):  #exp(-i theta/2 X)
    return np.array([[np.cos(theta/2), -1j*np.sin(theta/2)], [-1j*np.sin(theta/2), np.cos(theta/2)]])

def unitary_error_rxx(theta):          #exp(-i theta/2 X\otimes X)
    return np.cos(theta/2)*np.eye(4)-1j*np.sin(theta/2)*np.array([[0,0,0,1],[0,0,1,0],[0,1,0,0],[1,0,0,0]])

def unitary_error_rz(theta):  #exp(-i theta/2 Z)
    return np.array([[np.exp(-1j*theta/2), 0], [0, np.exp(1j*theta/2)]])

def unitary_error_rzz(theta):          #exp(-i theta/2 Z\otimes Z)
    return np.cos(theta/2)*np.eye(4)-1j*np.sin(theta/2)*np.array([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])

def unitary_error_ry(theta):  #exp(-i theta/2 Y)
    return np.array([[np.cos(theta/2), -np.sin(theta/2)], [np.sin(theta/2), np.cos(theta/2)]])

def unitary_error_ryy(theta):          #exp(-i theta/2 Y\otimes Y)
    return np.cos(theta/2)*np.eye(4)-1j*np.sin(theta/2)*np.array([[0,0,0,-1],[0,0,1,0],[0,1,0,0],[-1,0,0,0]])

# generate stochastic noise model
def get_noise_model(p=0):
    noise_model=NoiseModel()
    
    
    mu_1q=pauli_error([('Z',p/2),('I',1-p/2)])
    mu_2q= mu_1q.tensor(mu_1q)
    
    qerror_1q = mu_1q
    qerror_2q= mu_2q

    noise_model.add_all_qubit_quantum_error(qerror_1q, ['h','u2','u3','I','X','Y','Z'])
    noise_model.add_all_qubit_quantum_error(qerror_2q, ['cx'])
    
    return noise_model


# generate stochastic noise model
def get_noise_model2(p=0):
    noise_model=NoiseModel()
    
    
    mu_1q=pauli_error([('Z',p/4),('X',p/8),('Y',p/8),('I',1-p/2)])
    mu_2q= mu_1q.tensor(mu_1q)
    
    qerror_1q = mu_1q
    qerror_2q= mu_2q

    noise_model.add_all_qubit_quantum_error(qerror_1q, ['h','u2','u3','I','X','Y','Z'])
    noise_model.add_all_qubit_quantum_error(qerror_2q, ['cx'])
    
    return noise_model

# generate stochastic noise model
def get_noise_model3(p=0):
    noise_model=NoiseModel()
    
    
    mu_1q=pauli_error([('Z',p/6),('X',p/6),('Y',p/6),('I',1-p/2)])
    mu_2q= mu_1q.tensor(mu_1q)
    
    qerror_1q = mu_1q
    qerror_2q= mu_2q

    noise_model.add_all_qubit_quantum_error(qerror_1q, ['h','u2','u3','I','X','Y','Z'])
    noise_model.add_all_qubit_quantum_error(qerror_2q, ['cx'])
    
    return noise_model


# the model used to fit data
def model(a,d,nq=6):
    num_p= int(nq/2+1)
    return np.sum([a[i+2*num_p]*(a[i+num_p]**d)*(np.exp(-1j*a[i]*d)) for i in range(num_p)])

# the least-squre function for optimization
def func(a,x,y):
    est_y= np.array([model(a,d) for d in x])
    dif= np.sum(np.absolute(np.array(y)-est_y)**2)
    return dif

# generate starting point for least-square optimization by the Fourier tansform estimate
def generate_x0(phase_est_ft,damping_min=0.9,damping_max=1,coef_min=0.05,coef_max=1,nq=6):
    num_p=int(nq/2+1)
    x0_phase= list(phase_est_ft)
    x0_damping= [np.random.rand()*(damping_max-damping_min)+damping_min for i in range(num_p)]
    x0_coef= [np.random.rand()*(coef_max-coef_min)+coef_min for i in range(num_p)]
    return x0_phase + x0_damping+ x0_coef

# find the optimal estimate of parameters, such as phases, damping rates ...
def find_opt(data_list,L,nq=6,damping_min=0.8,damping_max=1,coef_min=0.05,coef_max=1):
    depth_list=[i for i in range(1,L+1)]
    data_len=len(data_list)
    num_p= int(nq/2+1)
    fourier_amp=list(np.absolute(np.fft.fft(data_list)))
    peaks=find_peaks(fourier_amp)[0]
    peak_vals=[fourier_amp[peak] for peak in peaks]
    loc=[peaks[peak_vals.index(x)] for x in np.sort(peak_vals)[-num_p::]]
    freq=np.fft.fftfreq(data_len)*2*pi
    phase_est_ft= np.sort(freq[loc])

    phase_bnds=[(min(phase_est_ft)-0.1,max(phase_est_ft)+0.1) for i in range(num_p)]
    damping_bnds= [(damping_min,damping_max) for i in range(num_p)]
    coef_bnds= [(coef_min,coef_max) for i in range(num_p)]
    bnds= phase_bnds+ damping_bnds+ coef_bnds
    
    x0_list=[generate_x0(phase_est_ft,damping_min,damping_max,coef_min,coef_max) for i in range(2)]

    res_list=[]
    for x0 in x0_list:
        res_list.append(minimize(func, x0, (depth_list,data_list),bounds=bnds))

    # est_fourier_dis=[np.mean(np.absolute(np.sort(res.x[0:num_p])-phase_est_ft)) for res in res_list]
    # min_est =min(est_fourier_dis)
    
    # index_filtered= np.where(np.array(est_fourier_dis)-min_est< 0.5*min_est)[0]
    # res_list_filtered= [res_list[index] for index in index_filtered]
    funcv_list=[res.fun for res in res_list]
    min_value= min(funcv_list)
    min_ind= funcv_list.index(min_value)
    return [res.x for res in res_list][min_ind],min_value


# find the optimal estimate of parameters, such as phases, damping rates ...
def find_opt_mp(data_list,L,mp_est,nq=6,damping_min=0.7,damping_max=1,coef_min=0.05,coef_max=1):
    depth_list=[i for i in range(1,L+1)]
    data_len=len(data_list)
    num_p= int(nq/2+1)
    fourier_amp=list(np.absolute(np.fft.fft(data_list)))
    peaks=find_peaks(fourier_amp)[0]
    peak_vals=[fourier_amp[peak] for peak in peaks]
    loc=[peaks[peak_vals.index(x)] for x in np.sort(peak_vals)[-num_p::]]
    freq=np.fft.fftfreq(data_len)*2*pi
    phase_est_ft= np.sort(freq[loc])

    phase_bnds=[(-pi,pi) for i in range(num_p)]
    damping_bnds= [(damping_min,damping_max) for i in range(num_p)]
    coef_bnds= [(coef_min,coef_max) for i in range(num_p)]
    bnds= phase_bnds+ damping_bnds+ coef_bnds
    
    x0_list=[generate_x0(phase_est_ft,damping_min,damping_max,coef_min,coef_max) for i in range(1)]+[mp_est]

    res_list=[]
    for x0 in x0_list:
        res_list.append(minimize(func, x0, (depth_list,data_list),bounds=bnds))

    # est_fourier_dis=[np.mean(np.absolute(np.sort(res.x[0:num_p])-phase_est_ft)) for res in res_list]
    # min_est =min(est_fourier_dis)
    
    # index_filtered= np.where(np.array(est_fourier_dis)-min_est< 0.5*min_est)[0]
    # res_list_filtered= [res_list[index] for index in index_filtered]
    funcv_list=[res.fun for res in res_list]
    min_value= min(funcv_list)
    min_ind= funcv_list.index(min_value)
    return [res.x for res in res_list][min_ind],min_value


# compute the phase estimation error
def error(est,nq=6):
    num_p= int(nq/2+1)
    dif=np.sort(est[0:num_p])-np.array(true_p)
    return np.mean(np.absolute(dif))


import scipy as sc
import scipy.linalg as la
import json
import math

#define matrix pencil data processing technique; this code is from the paper 'spectral quantum tomography' with some modifications
def matrix_pencil(time_series_data,L ,N_poles,cutoff=10**(-10)):
        """Computes a decomposition into exponentially decaying oscillations of a given time series.
        Input: time_series_data: a list of floats corresponding to the state of the system at fixed time intervals.
                NOTE: It is important this timeseries starts at t=0 (k=0), the method as implemented can't deal with timeshifts and may fail quietly
               L: A matrix pencil parameter. Set between 1/2 and 2/3 of len(time_series_data) 
               N_poles: The number of maximum possible poles the data can be decomposed into. Choosing this number too small will lead to bad fits so act with care.
               cutoff: a cut-off for the smallest possible relative amplitude a poles can contribute with, standard is 10^(-2).
        Ouput: poles: a scipy np.array of the poles (the oscillating bits).
                amplitudes: a scipy np.array of amplitudes corresponding to the poles.
                
                """
        
        #Compute the length of the data series and store it in N
        N = len(time_series_data)

        #Compute the Hankel matrix of the data
        Y = sc.matrix([time_series_data[i:L+i+1] for i in range(0,N-L)]) 
        
        #Take the singular value decomposition of the data Hankel matrix
        U,S,Vh = sc.linalg.svd(Y) 
        Vh =sc.matrix(Vh)
        U = sc.matrix(U)
        
        #The ESPRIT method includes a filter step that gets rid of small singular values.
        # Since for us the number of poles is known I just retain the N_poles largest singular values and corresponding right eigenspace.
        #If N_poles is so large that it starts to include nonsense values we let the cutoff parameter set the number of relevant poles instead.
        #We choose to retain only the singular values s such that s>s_max * cutoff where s_max is the largest singualr value
        Scutoff = S[S>cutoff*S[0]]
        if len(Scutoff)<N_poles:

            Sprime = sc.matrix([[S[i] if i==j else 0 for i in range(len(Scutoff))] for j in range(N-L)])
            Vhprime = Vh[0:len(Scutoff),:]
        else:
            Sprime = sc.matrix([[S[i] if i==j else 0 for i in range(N_poles)] for j in range(N-L)])
            Vhprime = Vh[0:N_poles,:]
        
        #Compute the shifted matrices for the matrix pencil.
        Vhprime1 = Vhprime[:,0:-1]
        Vhprime2 = Vhprime[:,1:]
        
        #Compute the solution of the matrix pencil (via SVD)
        Y = la.pinv(Vhprime1.H)*Vhprime2.H
        poles, vecs = sc.linalg.eig(Y)
        for i in range(len(poles)):
            amp= np.abs(poles[i])
            if amp>1:
                poles[i]= poles[i]/amp
        
        #Compute the amplitudes by least squares optimization
        Z = sc.matrix([poles**k for k in range(1,N+1)])
        amplitudes = la.lstsq(Z,sc.matrix(time_series_data).transpose())
        ampls = sc.array([a[0] for a in amplitudes[0]])
        
                
        #return the poles and amplitudes as scipy np.arrays
        return poles, ampls,amplitudes,S

def mp_est(data_list,nq=6):
    max_len= len(data_list)
    num_p= int(nq/2+1)
    poles, ampls, amplitudes, S= matrix_pencil(data_list,L= math.ceil(max_len*3/5),N_poles=num_p)
    x0_mp=list(np.angle(poles))+list(np.abs(poles))+list(np.abs(ampls))
    return np.angle(poles),x0_mp


In [ ]:
# actual process  of a circuit given a noise model
def circ_channel(noise_model,circ):
    circ_tar=circ.copy()
    circ_tar.save_superop()
    simulator= Aer.get_backend('aer_simulator_superop')
    noisy_sop= simulator.run(circ_tar,noise_model=noise_model).result()
    noisy_channel=noisy_sop.data()['superop']
    
    return noisy_channel

# simulation with a noisy channel
def sim_dm(L,dm_ini,noisy_channel):
    dm=dm_ini
    data=[]
    for i in range(L):
        dm= dm.evolve(noisy_channel)
        xexp=dm.expectation_value(np.array([[0,1],[1,0]]),[0])
        yexp=dm.expectation_value(np.array([[0,-1j],[1j,0]]),[0])
        data.append(xexp+1j*yexp)
    return data

# simulation with a noise model
def sim_dm_full(noise_model,L,nq=6):
    qc=sim_pre(nq=nq)
    backend = Aer.get_backend('aer_simulator_density_matrix')
    result = execute(qc, backend=backend,noise_model=noise_model,optimization_level=0).result()
    dm_ini=result.data(qc)['density_matrix'] 
    noisy_channel=circ_channel(noise_model,U)
    data=sim_dm(L=L,dm_ini=dm_ini,noisy_channel=noisy_channel)
    return data


In [ ]:
L_list=[3000]+[1000 for i in range(10)]
error_list_mp=[]
error_list=[]
noise_pro_list=np.linspace(0.0001,0.001,11)

for i,noise_pro in enumerate(noise_pro_list):
    L=L_list[i]
    
    noise_model= get_noise_model(noise_pro)
    data=sim_dm_full(noise_model,L=L)

    mp_phase,x0_mp= mp_est(data)
    error_list_mp.append(error(mp_phase))
    para,val=find_opt_mp(data,L=L,mp_est=x0_mp)
    error_list.append(error(para))

In [ ]:
L_list=[500 for i in range(10)]

noise_pro_list= list(np.linspace(0.001,0.01,11))[1::]

for i,noise_pro in enumerate(noise_pro_list):
    L=L_list[i]
    
    noise_model= get_noise_model(noise_pro)
    data=sim_dm_full(noise_model,L=L)

    mp_phase,x0_mp= mp_est(data)
    error_list_mp.append(error(mp_phase))
    para,val=find_opt_mp(data,L=L,mp_est=x0_mp)
    error_list.append(error(para))

In [ ]:
L_list=[1000 for i in range(9)]

noise_pro_list2= list(np.linspace(0.001,0.002,11))[1:-1]

error_list2=[]
error_list_mp2=[]
for i,noise_pro in enumerate(noise_pro_list2):
    L=L_list[i]
    
    noise_model= get_noise_model(noise_pro)
    data=sim_dm_full(noise_model,L=L)

    mp_phase,x0_mp= mp_est(data)
    error_list_mp2.append(error(mp_phase))
    para,val=find_opt_mp(data,L=L,mp_est=x0_mp)
    error_list2.append(error(para))

print(error_list2)